In [1]:
! pip install nltk
! pip install datasets
! pip install transformers[torch]
! pip install tokenizers
! pip install evaluate
! pip install rouge_score
! pip install sentencepiece
! pip install huggingface_hub

     ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
      --------------------------------------- 0.0/1.5 MB 330.3 kB/s eta 0:00:05
     - -------------------------------------- 0.0/1.5 MB 326.8 kB/s eta 0:00:05
     -- ------------------------------------- 0.1/1.5 MB 525.1 kB/s eta 0:00:03
     --- ------------------------------------ 0.1/1.5 MB 711.9 kB/s eta 0:00:02
     ----- ---------------------------------- 0.2/1.5 MB 831.5 kB/s eta 0:00:02
     ------- -------------------------------- 0.3/1.5 MB 1.0 MB/s eta 0:00:02
     ----------- ---------------------------- 0.4/1.5 MB 1.4 MB/s eta 0:00:01
     --------------- ------------------------ 0.6/1.5 MB 1.7 MB/s eta 0:00:01
     -------------------- ------------------- 0.8/1.5 MB 2.1 MB/s eta 0:00:01
     -------------------- ------------------- 0.8/1.5 MB 2.1 MB/s eta 0:00:

In [2]:
!pip install sentencepiece
!pip install datsets transformers[sentencepiece]

ERROR: Could not find a version that satisfies the requirement datsets (from versions: none)
ERROR: No matching distribution found for datsets


In [3]:
import nltk
import evaluate
import numpy as np
from datasets import load_dataset
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

c:\Users\praya\Downloads\NLP\health_NLP_chatbot\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from datasets import Dataset, DatasetDict

In [5]:
# Load the tokenizer, model, and data collator
MODEL_NAME = "google/flan-t5-base"

tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

tokenizer_config.json: 100%|██████████| 2.54k/2.54k [00:00<?, ?B/s]
spiece.model: 100%|██████████| 792k/792k [00:00<00:00, 2.64MB/s]
special_tokens_map.json: 100%|██████████| 2.20k/2.20k [00:00<?, ?B/s]
tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

In [ ]:
import pandas as pd

csv_file_path = "C:\Users\praya\Downloads\NLP\health_NLP_chatbot\datas\4.csv"

# Load the CSV file into a pandas DataFrame
df = pd.read_csv(csv_file_path)

# Display the first few rows of the DataFrame
print(df.head())


                                               Query  \
0  After aremok;IVF and intercourse on the same d...   
1  What are the chances of conception after treat...   
2  Is there any chance of pregnancy after an egg ...   
3  How can one conceive successfully?helo i am 25...   
4  How to confirm date of conception during pregn...   

                                              Answer  
0  u have good chances to get pregnant. ..u shoul...  
1  hello. it must be a tumour not sore like fibro...  
2  Hello. sometimes this does happen that when u ...  
3  Hi dear!I have read your question carefully an...  
4  hi there; I will suggest you the best possible...  


In [ ]:
# converting it to dataset
dataset = Dataset.from_pandas(df)
dataset_dict = DatasetDict({'train': dataset})


In [ ]:
print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['Query', 'Answer'],
        num_rows: 19
    })
})


In [ ]:
# DATA_NAME = "yahoo_answers_qa"
# yahoo_answers_qa = load_dataset(DATA_NAME)

# print(yahoo_answers_qa)

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'answer', 'nbestanswers', 'main_category'],
        num_rows: 87362
    })
})


In [ ]:
yahoo_answers_qa = dataset_dict["train"].train_test_split(test_size=0.3)
# TODO: Change back variable name

In [ ]:
yahoo_answers_qa

DatasetDict({
    train: Dataset({
        features: ['Query', 'Answer'],
        num_rows: 13
    })
    test: Dataset({
        features: ['Query', 'Answer'],
        num_rows: 6
    })
})

In [ ]:
# We prefix our tasks with "answer the question"
prefix = "Please answer this question: "

# Define the preprocessing function

def preprocess_function(examples):
   """Add prefix to the sentences, tokenize the text, and set the labels"""
   # The "inputs" are the tokenized answer:
   inputs = [prefix + doc for doc in examples["Query"]]
   model_inputs = tokenizer(inputs, max_length=128, truncation=True)

   # The "labels" are the tokenized outputs:
   labels = tokenizer(text_target=examples["Answer"],
                      max_length=512,
                      truncation=True)

   model_inputs["labels"] = labels["input_ids"]
   return model_inputs

In [ ]:
# Map the preprocessing function across our dataset
tokenized_dataset = yahoo_answers_qa.map(preprocess_function, batched=True)



Map:   0%|          | 0/13 [00:00<?, ? examples/s]

Map:   0%|          | 0/6 [00:00<?, ? examples/s]

FLAN-T5 Training and Fine-Tuning
Before diving into the training process, it is better to identify the metrics to evaluate the overall performance of the fine-tuning.

Good evaluation metrics are important in any deep learning and machine learning project to evaluate the performance of models, not only during training but also later in production.

Two of the most common metrics to evaluate the performance of a text generation model are BLEU and ROUGE, and in this case, to evaluate the quality of an answer by comparing it to a reference answer.

The focus of this tutorial is ROUGE, but this wikipedia article provides more information about the BLEU score.



In [ ]:
nltk.download("punkt", quiet=True)
metric = evaluate.load("rouge")



In [ ]:
def compute_metrics(eval_preds):
   preds, labels = eval_preds

   # decode preds and labels
   labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
   decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
   decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

   # rougeLSum expects newline after each sentence
   decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
   decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

   result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

   return result

## Training...

In [ ]:
# Global Parameters
L_RATE = 3e-4
BATCH_SIZE = 8
PER_DEVICE_EVAL_BATCH = 4
WEIGHT_DECAY = 0.01
SAVE_TOTAL_LIM = 3
NUM_EPOCHS = 3

# Set up training arguments
training_args = Seq2SeqTrainingArguments(
   output_dir="./results",
   evaluation_strategy="epoch",
   learning_rate=L_RATE,
   per_device_train_batch_size=BATCH_SIZE,
   per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
   weight_decay=WEIGHT_DECAY,
   save_total_limit=SAVE_TOTAL_LIM,
   num_train_epochs=NUM_EPOCHS,
   predict_with_generate=True,
   push_to_hub=False
)

In [ ]:
trainer = Seq2SeqTrainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_dataset["train"],
   eval_dataset=tokenized_dataset["test"],
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,No log,3.949146,0.045171,0.000000,0.041929,0.041644
2,No log,3.843748,0.038469,0.000000,0.038469,0.038469
3,No log,3.798832,0.052063,0.000000,0.051882,0.051882


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=6, training_loss=3.754373550415039, metrics={'train_runtime': 247.0485, 'train_samples_per_second': 0.158, 'train_steps_per_second': 0.024, 'total_flos': 6676384186368.0, 'train_loss': 3.754373550415039, 'epoch': 3.0})